In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 999
import csv
import gzip
import os
import scipy.io
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%reload_ext autoreload
%autoreload 2
np.set_printoptions(suppress=True)
import sys
sys.path.append('/home/roquero')
import pandas as pd
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects import r
from rpy2.robjects.conversion import localconverter
r("library(monocle)")

R[write to console]: Loading required package: Matrix

R[write to console]: Loading required package: Biobase

R[write to console]: Loading required package: BiocGenerics

R[write to console]: Loading required package: parallel

R[write to console]: 
Attaching package: ‘BiocGenerics’


R[write to console]: The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


R[write to console]: The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


R[write to console]: The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, ra

'monocle','DDRTree','irlba',...,'datasets','methods','base'


In [5]:
matrix_dir = "/home/roquero/store_new/data/gasperini/raw/"

## List of genes

In [3]:
table_genes = pd.read_table(os.path.join(matrix_dir, "GSE120861_at_scale_screen.genes.txt"), header=None)
table_genes.head()
table_genes.shape

,0
0,ENSG00000238009
1,ENSG00000237683
2,ENSG00000228463
3,ENSG00000237094
4,ENSG00000235373


(13135, 1)

## List of cells

In [4]:
table_cells = pd.read_table(os.path.join(matrix_dir, "GSE120861_at_scale_screen.cells.txt"), header=None)
table_cells.head()
table_cells.shape

,0
0,AAACCTGAGAGGTACC-1_1A_1_SI-GA-E2
1,AAACCTGAGTCAATAG-1_1A_1_SI-GA-E2
2,AAACCTGCAAACAACA-1_1A_1_SI-GA-E2
3,AAACCTGCACTTCTGC-1_1A_1_SI-GA-E2
4,AAACCTGCATGTAGTC-1_1A_1_SI-GA-E2


(207324, 1)

In [5]:
table_cells[1] = table_cells[0].apply(lambda u: u[-16:])
table_cells[2] = table_cells[0].apply(lambda u: u[:-16])

In [6]:
table_cells[1].value_counts()
table_cells[2].value_counts()

-1_1A_7_SI-GA-E8    7611
-1_1B_6_SI-GA-F7    7602
-1_1B_1_SI-GA-F2    7522
-1_1B_5_SI-GA-F6    7509
-1_1A_5_SI-GA-E6    7491
-1_1A_2_SI-GA-E3    7469
-1_1B_2_SI-GA-F3    7419
-1_1B_8_SI-GA-F9    7281
-1_1B_7_SI-GA-F8    7268
-1_1B_3_SI-GA-F4    7181
-1_1A_1_SI-GA-E2    7130
-1_1A_4_SI-GA-E5    7095
-1_1B_4_SI-GA-F5    7054
-1_1A_6_SI-GA-E7    6913
-1_2B_5_SI-GA-H6    6837
-1_2B_3_SI-GA-H4    6828
-1_2A_7_SI-GA-G8    6630
-1_2A_2_SI-GA-G3    6580
-1_2A_6_SI-GA-G7    6515
-1_2B_1_SI-GA-H2    6306
-1_2A_1_SI-GA-G2    6293
-1_2A_3_SI-GA-G4    6207
-1_2A_4_SI-GA-G5    6188
-1_2B_8_SI-GA-H9    6110
-1_2A_5_SI-GA-G6    6068
-1_2B_2_SI-GA-H3    5886
-1_2B_4_SI-GA-H5    5786
-1_2B_6_SI-GA-H7    5773
-1_2A_8_SI-GA-G9    5722
-1_2B_7_SI-GA-H8    5648
-1_1A_3_SI-GA-E4    5402
Name: 1, dtype: int64

CACATTTCAGTTTACG    5
CCTTACGCAATGTAAG    5
CTACCCAGTTCACGGC    5
GGAGCAAAGTTAACGA    5
GGGTCTGTCTCGCTTG    5
                   ..
TCACGAATCCAATGGT    1
ATTTCTGAGTTGCAGG    1
CCACGGATCGACAGCC    1
CCTATTAAGCGGATCA    1
GGGACCTAGCCAGTTT    1
Name: 2, Length: 181338, dtype: int64

## List of gRNA

In [7]:
table_gRNA = pd.read_table(os.path.join(matrix_dir, "GSE120861_grna_groups.at_scale.txt"), header=None)
table_gRNA.head()
table_gRNA.shape

,0,1
0,SH3BGRL3_TSS,AAACCGCTCCCGAGCACGGG
1,MTRNR2L8_TSS,AAATAGTGGGAAGATTCGTG
2,FAM83A_TSS,AACACACCACGGAGGAGTGG
3,ZNF593_TSS,AACAGCCCGGCCGGCCAAGG
4,ATPIF1_TSS,AACGAGAGACTGCTTGCTGG


(13189, 2)

In [59]:
table_gRNA[table_gRNA[0]=='chr16.1865_top_two']

,0,1
4182,chr16.1865_top_two,CTATGGGACTGGCAAACAGG
4183,chr16.1865_top_two,CTGCTTTCTGATGTGCACAG


## List of gRNA-gene pairs

In [8]:
table_gRNA_gene_pair = pd.read_table(os.path.join(matrix_dir, "GSE120861_gene_gRNAgroup_pair_table.at_scale.txt"))
table_gRNA_gene_pair.head(500)
table_gRNA_gene_pair.shape

/home/roquero/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,gRNAgroup.chr,gRNAgroup.start,gRNAgroup.stop,gRNAgroup,general_group,chr.targetgene,start.targetgene,stop.targetgene,ENSG.targetgene,targetgene_short_name,strand.targetgene,pairs
0,NTC,NTC,NTC,bassik_mch,NTC,chr10,28034777,28034778,ENSG00000150051,MKX,-,MKX:bassik_mch
1,NTC,NTC,NTC,bassik_mch,NTC,chr10,28287976,28287977,ENSG00000169126,ARMC4,-,ARMC4:bassik_mch
2,NTC,NTC,NTC,bassik_mch,NTC,chr10,28571017,28571018,ENSG00000150054,MPP7,-,MPP7:bassik_mch
3,NTC,NTC,NTC,bassik_mch,NTC,chr10,28821422,28821423,ENSG00000095787,WAC,+,WAC:bassik_mch
4,NTC,NTC,NTC,bassik_mch,NTC,chr10,28966271,28966272,ENSG00000095739,BAMBI,+,BAMBI:bassik_mch
5,NTC,NTC,NTC,bassik_mch,NTC,chr10,29577990,29577991,ENSG00000120563,LYZL1,+,LYZL1:bassik_mch
6,NTC,NTC,NTC,bassik_mch,NTC,chr10,29923900,29923901,ENSG00000197321,SVIL,-,SVIL:bassik_mch
7,NTC,NTC,NTC,bassik_mch,NTC,chr10,30348452,30348453,ENSG00000165757,KIAA1462,-,KIAA1462:bassik_mch
8,NTC,NTC,NTC,bassik_mch,NTC,chr10,30638247,30638248,ENSG00000107951,MTPAP,-,MTPAP:bassik_mch
9,NTC,NTC,NTC,bassik_mch,NTC,chr10,30722866,30722867,ENSG00000107968,MAP3K8,+,MAP3K8:bassik_mch


(1086564, 12)

In [9]:
len(table_gRNA_gene_pair.groupby('ENSG.targetgene'))

len(table_gRNA_gene_pair.groupby('targetgene_short_name'))

len(table_gRNA_gene_pair.groupby('gRNAgroup'))
len(table_gRNA_gene_pair.groupby('pairs'))

18026

18013

6585

1067388

In [10]:
table_gRNA_gene_pair[table_gRNA_gene_pair['targetgene_short_name']=='PRKCB']['ENSG.targetgene'].value_counts()
list_gRNA_groups_PRKCB = table_gRNA_gene_pair[(table_gRNA_gene_pair['targetgene_short_name']=='PRKCB') & (table_gRNA_gene_pair['general_group']!='NTC')]['gRNAgroup']
list_gRNA_groups_PRKCB

ENSG00000166501    62
Name: ENSG.targetgene, dtype: int64

975325        chr16.1822_top_two
975341        chr16.1856_top_two
975358        chr16.1857_top_two
975376     chr16.1863_second_two
975394        chr16.1863_top_two
975412        chr16.1865_top_two
975429        chr16.1866_top_two
975446     chr16.1866_second_two
975463        chr16.1867_top_two
975476        chr16.1897_top_two
1082333                PRKCB_TSS
Name: gRNAgroup, dtype: object

In [57]:
table_gRNA_gene_pair[table_gRNA_gene_pair['targetgene_short_name']=='PRKCB']

,gRNAgroup.chr,gRNAgroup.start,gRNAgroup.stop,gRNAgroup,general_group,chr.targetgene,start.targetgene,stop.targetgene,ENSG.targetgene,targetgene_short_name,strand.targetgene,pairs
5447,NTC,NTC,NTC,bassik_mch,NTC,chr16,23847322,23847323,ENSG00000166501,PRKCB,+,PRKCB:bassik_mch
23836,NTC,NTC,NTC,random_1,NTC,chr16,23847322,23847323,ENSG00000166501,PRKCB,+,PRKCB:random_1
42225,NTC,NTC,NTC,random_10,NTC,chr16,23847322,23847323,ENSG00000166501,PRKCB,+,PRKCB:random_10
60614,NTC,NTC,NTC,random_11,NTC,chr16,23847322,23847323,ENSG00000166501,PRKCB,+,PRKCB:random_11
79003,NTC,NTC,NTC,random_12,NTC,chr16,23847322,23847323,ENSG00000166501,PRKCB,+,PRKCB:random_12
97392,NTC,NTC,NTC,random_13,NTC,chr16,23847322,23847323,ENSG00000166501,PRKCB,+,PRKCB:random_13
115781,NTC,NTC,NTC,random_14,NTC,chr16,23847322,23847323,ENSG00000166501,PRKCB,+,PRKCB:random_14
134170,NTC,NTC,NTC,random_15,NTC,chr16,23847322,23847323,ENSG00000166501,PRKCB,+,PRKCB:random_15
152559,NTC,NTC,NTC,random_16,NTC,chr16,23847322,23847323,ENSG00000166501,PRKCB,+,PRKCB:random_16
170948,NTC,NTC,NTC,random_17,NTC,chr16,23847322,23847323,ENSG00000166501,PRKCB,+,PRKCB:random_17


In [58]:
table_gRNA_gene_pair[table_gRNA_gene_pair['gRNAgroup']=='chr16.1865_top_two']

,gRNAgroup.chr,gRNAgroup.start,gRNAgroup.stop,gRNAgroup,general_group,chr.targetgene,start.targetgene,stop.targetgene,ENSG.targetgene,targetgene_short_name,strand.targetgene,pairs
975398,chr16,23814819,23815225,chr16.1865_top_two,test_5k-untested,chr16,22825498,22825499,ENSG00000122254,HS3ST2,+,chr16.1865_top_two:ENSG00000122254
975399,chr16,23814819,23815225,chr16.1865_top_two,test_5k-untested,chr16,23083868,23083869,ENSG00000103404,USP31,-,chr16.1865_top_two:ENSG00000103404
975400,chr16,23814819,23815225,chr16.1865_top_two,test_5k-untested,chr16,23194036,23194037,ENSG00000166828,SCNN1G,+,chr16.1865_top_two:ENSG00000166828
975401,chr16,23814819,23815225,chr16.1865_top_two,test_5k-untested,chr16,23313591,23313592,ENSG00000168447,SCNN1B,+,chr16.1865_top_two:ENSG00000168447
975402,chr16,23814819,23815225,chr16.1865_top_two,test_5k-untested,chr16,23464500,23464501,ENSG00000168434,COG7,-,chr16.1865_top_two:ENSG00000168434
975403,chr16,23814819,23815225,chr16.1865_top_two,test_5k-untested,chr16,23521744,23521745,ENSG00000103365,GGA2,-,chr16.1865_top_two:ENSG00000103365
975404,chr16,23814819,23815225,chr16.1865_top_two,test_5k-untested,chr16,23568683,23568684,ENSG00000103353,UBFD1,+,chr16.1865_top_two:ENSG00000103353
975405,chr16,23814819,23815225,chr16.1865_top_two,test_5k-untested,chr16,23568664,23568665,ENSG00000103356,EARS2,-,chr16.1865_top_two:ENSG00000103356
975406,chr16,23814819,23815225,chr16.1865_top_two,test_5k-untested,chr16,23607611,23607612,ENSG00000004779,NDUFAB1,-,chr16.1865_top_two:ENSG00000004779
975407,chr16,23814819,23815225,chr16.1865_top_two,test_5k-untested,chr16,23652713,23652714,ENSG00000166847,DCTN5,+,chr16.1865_top_two:ENSG00000166847


## Results Table

In [6]:
table_results = pd.read_table(os.path.join(matrix_dir, "GSE120861_all_deg_results.at_scale.txt"))
table_results.head()
table_results.shape

/home/roquero/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (12,13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,pairs4merge,beta,intercept,fold_change.transcript_remaining,pvalue.raw,pvalue.empirical,pvalue.empirical.adjusted,ENSG,gene_short_name,gRNA_group,quality_rank_grna,target_site.chr,target_site.start,target_site.stop,target_gene.chr,target_gene.start,target_gene.stop,strand,site_type,outlier_gene
0,ACTB_TSS:ENSG00000008256,-0.114784,-3.432141,0.891559,2.746239e-01,0.261969034,0.831159512731444,ENSG00000008256,CYTH3,ACTB_TSS,control,chr7,5570339,5570340,chr7,6312274,6312275,-,TSS,False
1,ACTB_TSS:ENSG00000011275,-0.042450,-1.464958,0.958438,2.771529e-01,0.264361067,0.831814624048016,ENSG00000011275,RNF216,ACTB_TSS,control,chr7,5570339,5570340,chr7,5821369,5821370,-,TSS,False
2,ACTB_TSS:ENSG00000075618,0.034076,0.392365,1.034664,1.425945e-01,not_applicable,not_applicable,ENSG00000075618,FSCN1,ACTB_TSS,control,chr7,5570339,5570340,chr7,5632454,5632455,+,TSS,False
3,ACTB_TSS:ENSG00000075624,-0.437939,2.898258,0.645365,7.463645e-115,3.941e-06,0.000407265538461538,ENSG00000075624,ACTB,ACTB_TSS,control,chr7,5570339,5570340,chr7,5570339,5570340,-,selfTSS,False
4,ACTB_TSS:ENSG00000086232,-0.010209,1.154783,0.989843,6.253678e-01,0.611785198,0.947813794532952,ENSG00000086232,EIF2AK1,ACTB_TSS,control,chr7,5570339,5570340,chr7,6098860,6098861,-,TSS,False


(629515, 20)

In [8]:
table_results[table_results['ENSG']=='ENSG00000166501']

,pairs4merge,beta,intercept,fold_change.transcript_remaining,pvalue.raw,pvalue.empirical,pvalue.empirical.adjusted,ENSG,gene_short_name,gRNA_group,quality_rank_grna,target_site.chr,target_site.start,target_site.stop,target_gene.chr,target_gene.start,target_gene.stop,strand,site_type,outlier_gene
7691,BASSIK_MCH:ENSG00000166501,0.040011,-0.333695,1.040822,3.097857e-01,not_applicable,not_applicable,ENSG00000166501,PRKCB,bassik_mch,control,NTC,NTC,NTC,chr16,23847322,23847323,+,NTC,False
35116,CHR16.1822_TOP_TWO:ENSG00000166501,0.011942,-0.334934,1.012014,6.877255e-01,not_applicable,not_applicable,ENSG00000166501,PRKCB,chr16.1822_top_two,top_two,chr16,23286312,23286598,chr16,23847322,23847323,+,DHS,False
35129,CHR16.1856_TOP_TWO:ENSG00000166501,0.034294,-0.334341,1.034889,2.682205e-01,not_applicable,not_applicable,ENSG00000166501,PRKCB,chr16.1856_top_two,top_two,chr16,23738984,23739572,chr16,23847322,23847323,+,DHS,False
35142,CHR16.1857_TOP_TWO:ENSG00000166501,0.091915,-0.333969,1.096272,1.479578e-02,not_applicable,not_applicable,ENSG00000166501,PRKCB,chr16.1857_top_two,top_two,chr16,23741079,23741473,chr16,23847322,23847323,+,DHS,False
35155,CHR16.1863_SECOND_TWO:ENSG00000166501,0.028520,-0.335258,1.028930,3.322830e-01,not_applicable,not_applicable,ENSG00000166501,PRKCB,chr16.1863_second_two,second_two,chr16,23796060,23796501,chr16,23847322,23847323,+,DHS,False
35168,CHR16.1863_TOP_TWO:ENSG00000166501,0.022272,-0.337484,1.022522,4.437443e-01,not_applicable,not_applicable,ENSG00000166501,PRKCB,chr16.1863_top_two,top_two,chr16,23796060,23796501,chr16,23847322,23847323,+,DHS,False
35181,CHR16.1865_TOP_TWO:ENSG00000166501,-0.131349,-0.336060,0.876912,5.772509e-07,0.000587171,0.0362683489836888,ENSG00000166501,PRKCB,chr16.1865_top_two,top_two,chr16,23814819,23815225,chr16,23847322,23847323,+,DHS,False
35194,CHR16.1866_SECOND_TWO:ENSG00000166501,-0.742421,-0.338958,0.475960,3.322129e-71,3.941e-06,0.000407265538461538,ENSG00000166501,PRKCB,chr16.1866_second_two,second_two,chr16,23833191,23833694,chr16,23847322,23847323,+,DHS,False
35207,CHR16.1866_TOP_TWO:ENSG00000166501,-0.803588,-0.341382,0.447720,2.634908e-134,3.941e-06,0.000407265538461538,ENSG00000166501,PRKCB,chr16.1866_top_two,top_two,chr16,23833191,23833694,chr16,23847322,23847323,+,DHS,False
35220,CHR16.1867_TOP_TWO:ENSG00000166501,-0.243830,-0.334432,0.783621,2.880934e-12,0.000279793,0.0237470520938628,ENSG00000166501,PRKCB,chr16.1867_top_two,top_two,chr16,23837878,23838250,chr16,23847322,23847323,+,DHS,False


## Table reads

In [ ]:
mat = scipy.io.mmread(os.path.join(matrix_dir, "GSE120861_at_scale_screen.exprs.mtx"))
mat = pd.DataFrame.sparse.from_spmatrix(mat)

In [13]:
mat = mat.rename(columns=table_cells[0], index=table_genes[0]).transpose()

## Cell Dataset

In [14]:
_ = r("a <- readRDS('/home/roquero/store/data/gasperini/raw/GSE120861_at_scale_screen.cds.rds'); b <- pData(a) ")
_ = r("rm(a)")

In [15]:
colname_list = ['sample',
 'cell',
 'total_umis',
 'Size_Factor',
 'gene',
 'all_gene',
 'barcode',
 'read_count',
 'umi_count',
 'proportion',
 'guide_count',
 'sample_directory',
 'ko_barcode_file',
 'id',
 'prep_batch',
 'within_batch_chip',
 'within_chip_lane',
 'percent.mito']+list(list_gRNA_groups_PRKCB.values)
data = {colname:np.array(r('b${}'.format(colname))) for colname in colname_list}
table_crispr = pd.DataFrame(data=data)

In [44]:
colname_str = ['sample',
 'cell',
 'sample_directory',
 'ko_barcode_file',
 'id',
 'prep_batch',
 'within_batch_chip',
 'within_chip_lane']
table_crispr = table_crispr.astype({colname:'string' for colname in colname_str})
table_crispr.drop(columns=['gene', 'all_gene', 'barcode'], inplace=True)

In [45]:
table_crispr.to_pickle('table_crispr.pkl')

In [55]:
response = mat['ENSG00000166501']

In [56]:
response.to_pickle('response.pkl')